In [19]:
from helper_preprocessing import *

path_tests = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/tests.txt'
path_species = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/species.txt'
path_results = 'C:/Users/Simone/Desktop/Utilità tesi magistrale/data/results.txt'


tests, species, results = load_raw_data(path_tests, path_results, path_species)

prefiltered_data = prefilter(species, tests, results)

del tests, species, results

db = select_impute_features(prefiltered_data)
db.to_csv('no_repeated_experiments.csv')

Start! Wed Feb  3 15:29:08 2021
Loading raw data
tests table loaded!
species table loaded!
results table loaded!
Tests table dimensions:  (681605, 122)
Species table dimensions:  (27125, 15)
Results table dimensions:  (1006972, 137)
Prefiltering Step... Wed Feb  3 15:31:15 2021
Dimensions after prefiltering step:  (64341, 272)
Selection and Imputation Step... Wed Feb  3 15:31:28 2021
Features:
obs_duration_mean, obs_duration_unit, endpoint, effect, measurement, conc1_type, conc1_mean, conc1_unit, test_cas, test_location, exposure_type, control_type, media_type, application_freq_unit, class, tax_order, family, genus, species, 
Dimension after imputation step: (50160, 16)


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from general_helper import multiclass_encoding

def rep_exp_sensitivity(lout_exp):
    lst = [(1,i) for i in lout_exp]
    return (lst.count((1,1)) -1) / (len(lst) - 1)
   
def rep_exp_specificity(lout_exp):
    lst = [(0,i) for i in lout_exp]
    return (lst.count((0,0)) -1) / (len(lst) - 1)

imp_db = pd.read_csv('no_repeated_experiments.csv').drop(columns = 'Unnamed: 0')

imp_db['fish'] = imp_db['class'] + ' ' + imp_db['tax_order'] + ' ' + imp_db['family'] + ' ' +\
                        imp_db['genus'] + ' ' + imp_db['species']

imp_db['enc_conc'] = np.where(imp_db['conc1_mean'] > 1, 0, 1)

gdb = imp_db.groupby(['test_cas', 'obs_duration_mean', 'conc1_type', 'fish', 'exposure_type',
                     'control_type', 'media_type', 'application_freq_unit'])
gr = gdb.groups

specs = []
sens = []
for group in gr:
    lst = gdb.get_group(group).enc_conc.values.tolist()
    if len(lst) == 1:
        continue
    sen = rep_exp_sensitivity(lst)
    spec = rep_exp_specificity(lst)

    if sen > 0:
        sens.append(sen)
    if spec > 0:
        specs.append(spec)
       
    
print(np.mean(specs))
print(np.mean(sens))

0.9579867656199943
0.9387028706463307


In [3]:
import pandas as pd
import numpy as np
db_score_tot = pd.read_csv('no_repeated_experiments.csv').drop(['Unnamed: 0'],axis=1)

db_score_tot['fish'] = db_score_tot['class'] + ' ' + db_score_tot['tax_order'] + ' ' + db_score_tot['family'] + ' ' +\
                        db_score_tot['genus'] + ' ' + db_score_tot['species']

db_score_tot['score'] = np.where(db_score_tot['conc1_mean'].values > 1, 1, 0)

db_pair_num = pd.DataFrame(pd.pivot_table(data =db_score_tot, index=['test_cas', 'obs_duration_mean', 'conc1_type',
                                                                     'fish', 'exposure_type', 'control_type',
                                                                     'media_type', 'application_freq_unit'], 
                                          aggfunc='size'))
db_pair_num.rename(columns={ db_pair_num.columns[-1]: "count" }, inplace = True)


def count_pair(n):
    return sum(range(int(n)))
db_pair_num['count'] = db_pair_num['count'].apply(count_pair)


# count pairs for sensitivity and specificity
def se_sp(scores):
    if not np.any(scores):
        return None
    else:
        return np.sum([sum(scores[:i]) for i, score in enumerate(scores) if score==1])
            
def calculate_se_sp(db_score_tot):
    df = db_score_tot.groupby(['test_cas', 'obs_duration_mean', 'conc1_type',
                                 'fish', 'exposure_type', 'control_type',
                                 'media_type', 'application_freq_unit'])['score'].apply(np.array).reset_index()
    df['pair_se'] = df['score'].apply(se_sp)
    
    # switch the 0s and 1s 
    df['score'] = 1 - df['score']
    df['pair_sp'] = df['score'].apply(se_sp)
    
    df.drop(columns='score', inplace=True)
    return df

db_se_sp = calculate_se_sp(db_score_tot)

final_sp_se = pd.merge(db_se_sp, db_pair_num, how= 'left',on=['test_cas', 'obs_duration_mean', 'conc1_type',
                                 'fish', 'exposure_type', 'control_type',
                                 'media_type', 'application_freq_unit'])
final_sp_se['SE']  = final_sp_se['pair_se'].divide(final_sp_se['count'])
sensitivity = final_sp_se.loc[final_sp_se['SE'].between(0,1),'SE'].mean()

# specificity 
final_sp_se['SP']  = final_sp_se['pair_sp'].divide(final_sp_se['count'])
specificity = final_sp_se.loc[final_sp_se['SP'].between(0,1),'SP'].mean()

print('sensitivity_avg:',sensitivity,'\n', 'specificity_avg:', specificity,sep='')

sensitivity_avg:0.8596730691306851
specificity_avg:0.7820887477946522
